모델과 데이터에 관한 정보는 링크를 참조해주세요.

- [KLUE/Bert-base](https://huggingface.co/klue/bert-base)
- [NSMC](https://github.com/e9t/nsmc)

In [1]:
!pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import transformers
import datasets

print(np.__version__)
print(transformers.__version__)
print(datasets.__version__)

1.23.5
4.35.2
2.16.1


## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [3]:
import datasets
from datasets import load_dataset

nsmc_dataset = load_dataset('nsmc')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
nsmc_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

## STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [5]:
from transformers import AutoTokenizer

huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [6]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False)

In [7]:
hf_dataset = nsmc_dataset.map(transform, batched=True)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'attention_mask'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
hf_test_dataset = hf_dataset['test']

### validation data 생성
train데이터를 0.9:0.1 비율로 나누어서 생성함

In [10]:
hf_train_datasets = hf_dataset['train'].train_test_split(test_size=0.1)
hf_train_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'attention_mask'],
        num_rows: 135000
    })
    test: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [11]:
hf_train_dataset = hf_train_datasets['train']
hf_val_dataset = hf_train_datasets['test']

## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [12]:
from transformers import AutoModelForSequenceClassification

huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)
# WRAPPER_MODEL = xmp.MpModelWrapper(huggingface_model)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import os
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                            # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                  #learning_rate
    per_device_train_batch_size = 32,      # 각 device 당 batch size
    per_device_eval_batch_size = 32,       # evaluation 시에 batch size
    num_train_epochs = 3,                  # train 시킬 총 epochs
    weight_decay = 0.01,                   # weight decay
)

In [14]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc') # 그냥 똑같은거 써도 될것 같다.

def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

<ipython-input-14-644bf5c7ad2f>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'mrpc')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
# def train(model, training_arguments):
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
    # return trainer

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.251500,0.232896,0.905533,0.905678
2,0.177000,0.245723,0.906733,0.907222
3,0.121700,0.289982,0.909200,0.909345


TrainOutput(global_step=12657, training_loss=0.19259176565457964, metrics={'train_runtime': 9197.6063, 'train_samples_per_second': 44.033, 'train_steps_per_second': 1.376, 'total_flos': 1.065599774208e+17, 'train_loss': 0.19259176565457964, 'epoch': 3.0})

3 epoch 2:33:15 소요

In [17]:
eval_metrics = trainer.evaluate(hf_test_dataset)
print(eval_metrics)

{'eval_loss': 0.30192553997039795, 'eval_accuracy': 0.9058, 'eval_f1': 0.9071701683156608, 'eval_runtime': 369.2424, 'eval_samples_per_second': 135.412, 'eval_steps_per_second': 4.233, 'epoch': 3.0}


In [18]:
del huggingface_model

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

- [Data Collator](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)

- [Trainer.TrainingArguments 의 group_by_length](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.



In [19]:
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                            # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                  #learning_rate
    per_device_train_batch_size = 32,      # 각 device 당 batch size
    per_device_eval_batch_size = 32,       # evaluation 시에 batch size
    num_train_epochs = 3,                  # train 시킬 총 epochs
    weight_decay = 0.01,                   # weight decay
    group_by_length=True,
)

In [25]:
from transformers import DataCollatorWithPadding

trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=huggingface_tokenizer, pad_to_multiple_of=8),
)
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.249900,0.233359,0.907533,0.906201
2,0.177000,0.248778,0.909333,0.910231
3,0.114300,0.288121,0.913067,0.912776


TrainOutput(global_step=12657, training_loss=0.19326916315115356, metrics={'train_runtime': 9224.3713, 'train_samples_per_second': 43.905, 'train_steps_per_second': 1.372, 'total_flos': 1.065599774208e+17, 'train_loss': 0.19326916315115356, 'epoch': 3.0})

3 epoch 2:33:43 소요

In [26]:
eval_metrics_2 = trainer.evaluate(hf_test_dataset)
print(eval_metrics_2)

{'eval_loss': 0.30499252676963806, 'eval_accuracy': 0.90574, 'eval_f1': 0.9069808751258215, 'eval_runtime': 370.0993, 'eval_samples_per_second': 135.099, 'eval_steps_per_second': 4.223, 'epoch': 3.0}


## 회고

NSMC 데이터셋과 klue/bert-base를 활용하여 한국어 데이터셋에 대한 Bert 파인튜닝을 수행하였다.

Bucketing을 적용하면 학습시간이 줄어들고 성능이 감소하는 trade-off 현상을 기대하였으나 학습시간은 사실상 같았고 test accuracy 역시 살짝 감소하였다.

학습 중의 validation dataset에 대해서는 오히려 이전보다 train_accuracy가 좋게 나오기도하였다.

두 경우 모두 test accuracy가 90.58%, 90.574%로 90%를 넘겼다.

GPT 선생님께서는 훈련 시간이 줄어들지 않고 오히려 늘어난 이유에 대해 다음과 같은 4가지 이유를 드셨다.
1. 배치 구성의 복잡성 증가
2. 데이터 로딩과 전처리 오버헤드
3. GPU 사용 효율성의 감소
4. 하드웨어 및 네트워크 병목

그러고 다음과 같은 해결방안을 주셨다.
1. 배치 크기 재조정: GPU 메모리 사용을 최적화하기 위해 배치 크기를 조정해보세요.
2. 프로파일링: 훈련 과정을 프로파일링하여 병목 지점을 파악하고, 데이터 로딩, 전처리, 모델 계산 등 각 단계에서의 시간 소요를 분석하세요.
3. 하드웨어 및 네트워크 설정 확인: 데이터 로딩 속도와 네트워크 대역폭 등을 점검하여 하드웨어와 네트워크가 훈련 성능에 영향을 미치고 있는지 확인하세요.

한번 훈련하는데 2시간 30분이나 걸리기 때문에 + 게다가 너무 오래 걸리거나 메모리 문제로 colab A100을 사용하였기 때문에 당장 추가 실험보다는 여기서 마치는게 맞을 것 같다. + 아직 지금 저것들을 분석할 충분한 능력도 없다.
